# Hydrological and Environmental Measurements in Bay of Biscay and Adjacent Waters Exploration with `mlcroissant`
This notebook demonstrates loading and exploring the FAIR² hydrological and environmental dataset using the `mlcroissant` library. The dataset is described using a Croissant schema and contains multiple record sets representing water quality, sediment composition, biotic abundance, and contaminant concentrations across the Bay of Biscay region.

### Dataset Source
The dataset Croissant schema is accessible at:
```
https://sen.science/doi/10.71728/senscience.h3h6-tjan/fair2.json
```

In [ ]:
# Ensure `mlcroissant` library is installed
!pip install mlcroissant

## 1. Data Loading
Load metadata and records from the dataset using `mlcroissant`.

In [ ]:
import mlcroissant as mlc
import pandas as pd

# Define the Croissant schema URL
croissant_url = "https://sen.science/doi/10.71728/senscience.h3h6-tjan/fair2.json"

# Load the dataset metadata
dataset = mlc.Dataset(croissant_url)
metadata = dataset.metadata

print(f"\033[1m{metadata.name}\033[0m\n{metadata.description}\n")
print(f"License: {metadata.license}")
print(f"Date Published: {metadata.datePublished}")
print(f"Collection Timeframe: {metadata.dataCollectionTimeframe}")
print(f"Data Biases: {metadata.dataBiases if hasattr(metadata, 'dataBiases') else 'N/A'}")
print(f"Data Use Cases: {metadata.dataUseCases if hasattr(metadata, 'dataUseCases') else 'N/A'}")

## 2. Data Overview
Let's review available record sets and their fields. All references will use the unique `@id` for each record set and field, as defined by the Croissant schema.

In [ ]:
# List all record sets in the dataset using their `@id`
record_sets = list(dataset.record_sets)

print("Available record sets (by @id):")
for rs in record_sets:
    print(f"- {rs['@id']}   ({rs['name']})")

# Show fields/columns in each record set
for rs in record_sets:
    print(f"\nRecord Set '{rs['name']}' (@id: {rs['@id']}):")
    if 'field' in rs and rs['field']:
        for field in rs['field']:
            # The field may be a dict or a uri string; handle both
            field_obj = field if isinstance(field, dict) else dataset.field_by_id(field)
            if field_obj is not None:
                print(f"  - {field_obj['@id']}   ({field_obj.get('name', '')})")
    else:
        print("  No fields listed in record set.")

# For demonstration, show a few example records from the first record set (if present)
if record_sets:
    print(f"\nExample records from {record_sets[0]['@id']}:")
    for idx, rec in enumerate(dataset.records(record_set=record_sets[0]['@id'])):
        if idx >= 3:
            break
        print(rec)

## 3. Data Extraction
Load data from selected record sets into Pandas DataFrames for further analysis.

Below, we extract all available record sets using their `@id`. The DataFrames are stored in a dictionary with record set `@id` as keys.

In [ ]:
# Prepare a dictionary to hold our DataFrames
dataframes = {}

# List of record set @ids
record_set_ids = [rs['@id'] for rs in record_sets]

for record_set_id in record_set_ids:
    try:
        records = list(dataset.records(record_set=record_set_id))
        if records:
            df = pd.DataFrame(records)
            dataframes[record_set_id] = df
            print(f"Loaded {len(df)} records for record set '@id': {record_set_id}")
        else:
            print(f"No records found in record set '@id': {record_set_id}")
    except Exception as e:
        print(f"Failed to load record set {record_set_id}: {e}")

# Display columns of the first DataFrame as an example
if dataframes:
    first_rs_id = next(iter(dataframes))
    print(f"\nColumns in DataFrame for record set '@id': {first_rs_id}")
    print(dataframes[first_rs_id].columns.tolist())
    display(dataframes[first_rs_id].head(3))

## 4. Exploratory Data Analysis (EDA)
Let's select a record set and numeric field (referenced by `@id` per Croissant) for analysis. We'll filter, normalize, and group data based on these fields.

*(Edit the `selected_record_set_id`, `numeric_field_id`, and `group_field_id` variables below to explore other parts of the dataset. You can find these `@id`s from previous outputs.)*

In [ ]:
# Specify the record set and fields (all by '@id')
# (Replace these values with the @id of your interest, using the overview in previous cells)
selected_record_set_id = ''
numeric_field_id = ''
group_field_id = ''

# --- Fallback: Attempt to auto-select if not set ---
if not selected_record_set_id:
    # Select the first DataFrame with numeric fields
    for rs_id, df in dataframes.items():
        for col in df.columns:
            if pd.api.types.is_numeric_dtype(df[col]):
                selected_record_set_id = rs_id
                numeric_field_id = col
                break
        if selected_record_set_id:
            break
if not group_field_id and selected_record_set_id:
    # Pick the first non-numeric field as group_field_id
    df = dataframes[selected_record_set_id]
    for col in df.columns:
        if not pd.api.types.is_numeric_dtype(df[col]):
            group_field_id = col
            break

if not selected_record_set_id or not numeric_field_id:
    print("Unable to auto-select fields. Please set selected_record_set_id and numeric_field_id to valid @id values from your dataset.")
else:
    df = dataframes[selected_record_set_id]
    print(f"Using record set '@id': {selected_record_set_id}")
    print(f"Numeric field for EDA: '{numeric_field_id}'")
    # Filtering
    if pd.api.types.is_numeric_dtype(df[numeric_field_id]):
        threshold = df[numeric_field_id].quantile(0.50)  # Median as threshold for filtering
        filtered_df = df[df[numeric_field_id] > threshold].copy()
        print(f"Filtered {len(filtered_df)}/{len(df)} records with {numeric_field_id} > {threshold}")
        # Normalization
        filtered_df[f"{numeric_field_id}_normalized"] = (
            (filtered_df[numeric_field_id] - filtered_df[numeric_field_id].mean()) /
            (filtered_df[numeric_field_id].std() if filtered_df[numeric_field_id].std() > 1e-9 else 1)
        )
        display(filtered_df[[numeric_field_id, f"{numeric_field_id}_normalized"]].head())
    else:
        print(f"Field '{numeric_field_id}' is not numeric, cannot filter or normalize.")

    # Grouping
    if group_field_id and group_field_id in filtered_df.columns:
        grouped_df = filtered_df.groupby(group_field_id)[numeric_field_id].mean().reset_index()
        print(f"\nMean of '{numeric_field_id}' grouped by '{group_field_id}' (by @id):")
        display(grouped_df.head())
    else:
        print(f"No suitable group field available for grouping.")

## 5. Visualization
Let's visualize the distribution of the selected numeric field, and (if applicable) compare it across the grouping field.

*(The visualizations reference all fields by their `@id`, in alignment with the Croissant schema.)*

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

if selected_record_set_id and numeric_field_id and selected_record_set_id in dataframes:
    df = dataframes[selected_record_set_id]
    plt.figure(figsize=(8, 5))
    sns.histplot(df[numeric_field_id].dropna(), bins=30, kde=True)
    plt.title(f"Distribution of field '@id': {numeric_field_id}")
    plt.xlabel(numeric_field_id)
    plt.ylabel('Count')
    plt.show()

    # Boxplot per group field if available
    if group_field_id and group_field_id in df.columns:
        plt.figure(figsize=(10, 5))
        sns.boxplot(x=group_field_id, y=numeric_field_id, data=df)
        plt.title(f"Boxplot of '{numeric_field_id}' by group '@id': {group_field_id}")
        plt.xlabel(group_field_id)
        plt.ylabel(numeric_field_id)
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.show()

## 6. Conclusion
This notebook demonstrated how to load and explore a FAIR² environmental dataset using the `mlcroissant` library and the Croissant schema.

- Each record set, field, and data column was referenced and processed according to its unique `@id` identifier, consistent with best practices for schema-based data.
- We loaded all available record sets, previewed records, normalized numeric fields, grouped results, and visualized distributions and relationships between fields.
- This workflow facilitates reproducible, schema-driven data analysis for complex, multi-table open datasets.

**Next steps:** Try switching `record_set_id`, `numeric_field_id`, and `group_field_id` to other values (see Data Overview) to analyze different data tables and fields from the FAIR² dataset.